Script per provar coses

# Llibreries

In [1]:
import os
import glob
from datetime import datetime

import pandas as pd
import numpy as np
from decimal import Decimal
import collections
import time

from sklearn import datasets, linear_model
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

import statsmodels.formula.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from brokenaxes import brokenaxes

from scipy.stats import ttest_ind, ttest_ind_from_stats
from scipy.special import stdtr

from functools import reduce

# Input files (RAW data)

In [2]:
path = r'C:/CicloHidrico/'
os.chdir(path)

estID = pd.read_csv("estaciones.csv", sep=';', engine='python', index_col=0)                   # estaciones con ID number
estRang = pd.read_excel("EstacionesTermoPluvio.xlsx", header=1, sep=';', usecols=range(1, 10)) # estaciones con rango de fecha

In [3]:
path2 = r'C:/CicloHidrico/Datos'
os.chdir(path2)

precipitacion = pd.read_csv("precipitacion.csv", sep=';', engine='python', index_col=0)   # precipitacion total diaria

In [4]:
t_max = pd.read_csv("temperatura_max.csv", sep=';', index_col=0)      # temperaturas màximas diarias
t_min = pd.read_csv("temperatura_min.csv", sep=';', index_col=0)      # temperaturas màximas diarias

# Dades d'estacions

## Campos duplicados

In [5]:
# estID[estID.duplicated('CodigoEst')==True]
# estID.duplicated('Tipo').value_counts
duplicateRowsDF = estID.Estacion[estID.duplicated(['Altitud','Xutm','Yutm'])==True]
# duplicateRowsDF = estID.Estacion[estID.duplicated(['Estacion'])==True]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
260                            XABIA
263    XABIA, AJUNTAMENT, AUTOMATICA
306                       GANDIA, HE
307                           GANDIA
507            BUNYOL, C.P. SAN LUIS
513                      CARLET, SMN
552              ALDAIA, HORT NOTARI
613              CHELVA, EL CALVARIO
666                 MASSAMAGRELL, HS
683             SEGORBE, LA ARTELINA
711                  NULES, PIUS XII
723        VILA-REAL, CENTRO C. AGR.
832          CASTELLFORT, AUTOMATICA
Name: Estacion, dtype: object


## Merge dataframes

In [6]:
estIDabc=estID.sort_values('Estacion').reset_index(drop=True)                              # ordena estaciones alfabéticamente

estRang = estRang.rename(columns={'Longitud': 'Long', 'Latitud': 'Lat', 'Provincia':'Prov'}) # evita columnas con mismo nombre
estaciones = estIDabc.join(estRang)                                                            # concatena DFs

estaciones.head()

,CodigoEst,Estacion,Tipo,Longitud,Latitud,Altitud,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,ESTACIÓN,Long,Lat,Altura,Prov,Cobertura Temperatura,Cobertura Precipitación,Temp,Pluv
0,1387,A CORUNYA (ESTACION COMPLETA),INMman,#¡CAMPO!,43.37,58,59149,4791283,AC,1,A CORUNYA (ESTACION COMPLETA),-8.42,43.37,58,AC,<->,1930-10-01<->2010-10-31,NO,SI
1,8359,ABABUJ,INMman,#¡CAMPO!,40.56,1368,686597,4491216,TE,3,ABABUJ,-1.28,40.56,1368,TE,<->,<->,NO,NO
2,7250,"ABANILLA, CHS",INMman,#¡CAMPO!,38.20,222,672506,4229911,MU,2,"ABANILLA, CHS",-1.03,38.20,222,MU,1958-01-01<->2010-10-31,1958-01-01<->2010-10-31,SI,SI
3,7150,ABARAN (SIERRA DE LA PILA),INMman,#¡CAMPO!,38.25,300,646400,4235009,MU,3,ABARAN (SIERRA DE LA PILA),-1.32,38.25,300,MU,2007-10-01<->2010-10-31,1961-04-01<->2010-10-31,SI,SI
4,7151,ABARAN (SIERRA DEL ORO),INMman,#¡CAMPO!,38.18,400,638133,4227865,MU,3,ABARAN (SIERRA DEL ORO),-1.42,38.18,400,MU,2007-10-01<->2010-10-31,1957-04-01<->2010-10-31,SI,SI


In [91]:
estaciones[estaciones['Cobertura Precipitación']=="<->"]

,CodigoEst,Estacion,Tipo,Longitud,Latitud,Altitud,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,ESTACIÓN,Long,Lat,Altura,Prov,Cobertura Temperatura,Cobertura Precipitación,Temp,Pluv
1,8359,ABABUJ,INMman,#¡CAMPO!,40.56,1368,686597,4491216,TE,3,ABABUJ,-1.28,40.56,1368,TE,<->,<->,NO,NO
17,8360,AGUILAR DE ALFAMBRA,INMman,#¡CAMPO!,40.59,1302,686254,4495527,TE,3,AGUILAR DE ALFAMBRA,-0.80,40.59,1302,TE,<->,<->,NO,NO
21,8025D,"ALACANT, FITOSANITARIA",INMman,#¡CAMPO!,38.35,10,720220,4247726,A,1,"ALACANT, FITOSANITARIA",-0.48,38.35,10,A,<->,<->,NO,NO
31,8354,"ALBARRACIN, CHJ",INMman,#¡CAMPO!,40.41,1182,632108,4474264,TE,3,"ALBARRACIN, CHJ",-1.44,40.41,1182,TE,<->,<->,NO,NO
32,8354E,"ALBARRACIN, PFE",INMman,#¡CAMPO!,40.44,1350,631108,4475228,TE,3,"ALBARRACIN, PFE",-1.47,40.44,1350,TE,<->,<->,NO,NO
37,8460,ALCALA DE LA SELVA,INMman,#¡CAMPO!,40.37,1404,693458,4471641,TE,3,ALCALA DE LA SELVA,-0.72,40.37,1404,TE,<->,<->,NO,NO
55,8364A,"ALFAMBRA, CHJ",INMman,#¡CAMPO!,40.41,1047,632108,4474264,TE,3,"ALFAMBRA, CHJ",-1.44,40.41,1047,TE,<->,<->,NO,NO
56,8364,"ALFAMBRA, PFE-DGA",INMman,#¡CAMPO!,40.44,1047,629650,4477300,TE,3,"ALFAMBRA, PFE-DGA",-1.47,40.44,1047,TE,<->,<->,NO,NO
67,9557,ALIAGA,INMman,#¡CAMPO!,40.68,1120,694333,4505332,TE,3,ALIAGA,-0.70,40.68,1120,TE,<->,<->,NO,NO
101,8302D,"ALZIRA, VALLETES DEL GALLO",INMman,#¡CAMPO!,39.15,200,720367,4336593,V,1,"ALZIRA, VALLETES DEL GALLO",-0.45,39.15,200,V,<->,<->,NO,NO


In [ ]:
# # crear columnas vacias

# precipitacion["Estacion"] = ""
# precipitacion["Provincia"] = ""
# precipitacion["Xutm"] = ""
# precipitacion["Yutm"] = ""
# precipitacion["Altitud"] = ""
# precipitacion["km_costa"] = ""    # km a la costa: 1 = costa a < 20km, 2 = de 20 a 40 km, 3 = + de 40 km 

In [116]:
a = precipitacion[precipitacion.index.duplicated(keep='first')==False]

In [7]:
dataPP = pd.merge(precipitacion,
                estaciones[['Estacion','CodigoEst','Tipo','Xutm','Yutm','Provincia','Umbral_20_40_km_costa']],
                on='CodigoEst')
dataPP.describe(include='all')

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Provincia,Umbral_20_40_km_costa
count,9895905,9895905,9.895905e+06,9895905,9895905,9895905,9.895905e+06,9.895905e+06,9895905,9.895905e+06
unique,840,41641,NaN,3,835,4,NaN,NaN,29,NaN
top,9981A,1994-09-15,NaN,S,"TORTOSA, OBSER. DEL EBRO",INMman,NaN,NaN,V,NaN
freq,36012,507,NaN,9389160,36012,9734857,NaN,NaN,2717440,NaN
mean,NaN,NaN,-3.226154e+00,NaN,NaN,NaN,6.568422e+05,4.349168e+06,NaN,2.170715e+00
std,NaN,NaN,2.210382e+01,NaN,NaN,NaN,1.123354e+05,1.112732e+05,NaN,8.670858e-01
min,NaN,NaN,-9.990000e+01,NaN,NaN,NaN,3.344300e+04,4.075366e+06,NaN,1.000000e+00
25%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,6.209690e+05,4.270236e+06,NaN,1.000000e+00
50%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,6.913740e+05,4.343742e+06,NaN,2.000000e+00
75%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,7.226450e+05,4.418167e+06,NaN,3.000000e+00


In [8]:
from functools import reduce

data = reduce(lambda x,y: pd.merge(x,y, on=['CodigoEst','Fecha'], how='outer'), [dataPP, t_max, t_min])

In [162]:
len(data[data.Provincia =='A'])

1645766

In [163]:
data.tail(500)

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
10045838,8050X,1998-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.5,S,11.3,S
10045839,8050X,1999-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.9,S,13.0,S
10045840,8050X,1998-10-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.9,S,16.4,S
10045841,8050X,1998-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.6,S,21.6,S
10045842,8050X,1998-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.4,S,2.0,S
10045843,8050X,1998-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.2,S,22.6,S
10045844,8050X,1999-03-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.4,S,3.4,S
10045845,8050X,1999-03-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,S,11.9,S
10045846,8050X,1997-08-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.8,S,20.4,S
10045847,8050X,1999-06-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.3,S,19.2,S


# Cleaning

## Drop rows without valid precipitation

In [9]:
data.dropna(subset=['Precipitacion','ValidPP'],inplace=True) # eliminar files sense precipitació
# data.tail(200)

In [10]:
data.Precipitacion[(data.Precipitacion<0) & (data.ValidPP=='N')]#.describe()

7928      -99.9
7929      -99.9
7930      -99.9
7931      -99.9
7932      -99.9
7933      -99.9
7934      -99.9
7935      -99.9
7936      -99.9
7937      -99.9
7938      -99.9
7939      -99.9
7940      -99.9
7941      -99.9
7942      -99.9
7943      -99.9
7944      -99.9
7945      -99.9
7946      -99.9
7947      -99.9
7948      -99.9
7949      -99.9
7950      -99.9
7951      -99.9
7952      -99.9
7953      -99.9
7954      -99.9
7955      -99.9
7956      -99.9
7957      -99.9
           ... 
9893726    -0.3
9893759    -0.3
9893809    -0.3
9893928    -0.4
9893971    -0.3
9893988    -0.3
9893998    -0.4
9894062    -0.4
9894081    -0.4
9894131    -0.4
9894139    -0.4
9894175    -0.4
9894272    -0.3
9894310    -0.4
9894326    -0.3
9894355    -0.4
9894466    -0.3
9894476    -0.3
9894560    -0.3
9894631    -0.3
9894648    -0.4
9894770    -0.3
9894775    -0.3
9894827    -0.3
9894837    -0.3
9894870    -0.3
9894910    -0.3
9894973    -0.3
9895001    -0.4
9895038    -0.3
Name: Precipitacion, Len

In [196]:
%run ~/.ipython/profile_default/ipython_config.py

NameError: name 'null' is not defined

In [9]:
data.dropna(how='all') # drop rows where all fields are empty

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
0,7263,1969-11-01,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
1,7263,1969-11-02,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
2,7263,1969-11-03,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
3,7263,1969-11-04,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
4,7263,1969-11-05,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
5,7263,1969-11-06,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
6,7263,1969-11-07,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
7,7263,1969-11-08,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
8,7263,1969-11-09,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
9,7263,1969-11-10,0.1,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN


In [201]:
data.isnull().sum()    # no NaN data in Precipitacion, only in Temperatures

CodigoEst                      0
Fecha                          0
Precipitacion                  0
ValidPP                        0
Estacion                       0
Tipo                           0
Xutm                           0
Yutm                           0
Provincia                      0
Umbral_20_40_km_costa          0
Tmaxima                  6537966
ValidTmax                6537966
Tminima                  6538581
ValidTmin                6538581
dtype: int64

In [202]:
data.dtypes

CodigoEst                 object
Fecha                     object
Precipitacion            float64
ValidPP                   object
Estacion                  object
Tipo                      object
Xutm                     float64
Yutm                     float64
Provincia                 object
Umbral_20_40_km_costa    float64
Tmaxima                  float64
ValidTmax                 object
Tminima                  float64
ValidTmin                 object
dtype: object

In [449]:
data[(data.ValidPP == 'A') & (data.Precipitacion>0)].describe(include='all')

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
count,6613,6613,6613.000000,6613,6613,6613,6613.000000,6.613000e+03,6613,6613.000000,2843.000000,2843,2838.000000,2838
unique,454,1840,NaN,1,453,1,NaN,NaN,21,NaN,NaN,2,NaN,2
top,8029D,2003-05-08,NaN,A,"TORRE MACANES, AYUNTAMIENTO",INMman,NaN,NaN,V,NaN,NaN,S,NaN,S
freq,226,62,NaN,6613,226,6613,NaN,NaN,2678,NaN,NaN,2753,NaN,2735
mean,NaN,NaN,24.538817,NaN,NaN,NaN,693979.927718,4.334011e+06,NaN,1.925147,15.699859,NaN,9.368252,NaN
std,NaN,NaN,29.477824,NaN,NaN,NaN,63799.684492,9.432810e+04,NaN,0.871505,7.287064,NaN,5.842070,NaN
min,NaN,NaN,0.100000,NaN,NaN,NaN,166863.000000,4.075366e+06,NaN,1.000000,-5.000000,NaN,-10.000000,NaN
25%,NaN,NaN,7.400000,NaN,NaN,NaN,678011.000000,4.276487e+06,NaN,1.000000,11.000000,NaN,6.000000,NaN
50%,NaN,NaN,15.000000,NaN,NaN,NaN,706063.000000,4.322770e+06,NaN,2.000000,16.000000,NaN,10.000000,NaN
75%,NaN,NaN,30.000000,NaN,NaN,NaN,725072.000000,4.389576e+06,NaN,3.000000,20.500000,NaN,13.500000,NaN


In [11]:
# Input
ValidPP = ['N', 'A']          # casos con validez diferente a 'S'
value = [-99.9, -0.4,-0.3, 0] # valores concretos con precipitación menor que 0

print ('--- Validez de la precipitación ---')
print("\n")

for each1 in ValidPP:
    lentot = len(data[(data.ValidPP == each1)])
    lenpos = len(data[(data.ValidPP == each1) & (data.Precipitacion > 0)])
    print ('Hay '+str(lentot)+' casos con validez '+ each1+' , de los cuales '+str(lenpos) +' tienen precipitacion > 0')
    for each2 in value:
        lenneg = len(data[(data.ValidPP == each1) & (data.Precipitacion == each2)])
        print('    Cuando la validez es '+ each1 +' y la cantidad es ' + str(each2)+' mm, hay '+ str(lenneg) +' casos' )
    print("\n")
    

--- Validez de la precipitación ---


Hay 497101 casos con validez N , de los cuales 3256 tienen precipitacion > 0
    Cuando la validez es N y la cantidad es -99.9 mm, hay 450924 casos
    Cuando la validez es N y la cantidad es -0.4 mm, hay 4752 casos
    Cuando la validez es N y la cantidad es -0.3 mm, hay 34052 casos
    Cuando la validez es N y la cantidad es 0 mm, hay 4117 casos


Hay 9644 casos con validez A , de los cuales 6613 tienen precipitacion > 0
    Cuando la validez es A y la cantidad es -99.9 mm, hay 1624 casos
    Cuando la validez es A y la cantidad es -0.4 mm, hay 1033 casos
    Cuando la validez es A y la cantidad es -0.3 mm, hay 3 casos
    Cuando la validez es A y la cantidad es 0 mm, hay 371 casos




In [14]:
data_raw = data
data_raw.head()

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
0,7263,1969-11-01,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
1,7263,1969-11-02,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
2,7263,1969-11-03,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
3,7263,1969-11-04,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
4,7263,1969-11-05,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN


In [271]:
data_raw[(data_raw.CodigoEst=='7263') & (data_raw.ValidPP=='A')]

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
11655,7263,2001-10-10,31.8,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
11825,7263,2002-03-29,4.4,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
11838,7263,2002-04-11,24.1,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
12055,7263,2002-11-14,3.5,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
12114,7263,2003-01-12,14.8,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
13666,7263,2007-04-14,1.8,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
15262,7263,2014-11-04,9.0,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
16379,7263,2016-12-17,15.0,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
17183,7263,2013-11-27,31.8,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
17393,7263,2017-11-29,3.8,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN


In [ ]:
df_new = df.drop(df[(df['col_1'] == 1.0) & (df['col_2'] == 0.0)].index)
df = df.drop(df[df["R1"] == 1].index)
df = df.drop(df.index[np.where(df["R1"] == 1)[0]])

In [464]:
data_raw.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9895905 entries, 0 to 9895904
Data columns (total 14 columns):
CodigoEst                object
Fecha                    object
Precipitacion            float64
ValidPP                  object
Estacion                 object
Tipo                     object
Xutm                     float64
Yutm                     float64
Provincia                object
Umbral_20_40_km_costa    float64
Tmaxima                  float64
ValidTmax                object
Tminima                  float64
ValidTmin                object
dtypes: float64(6), object(8)
memory usage: 5.2 GB


In [51]:
# elimina validez N (linea 1) y elimina validez A solo en el caso que pp = -99.9 mm 

data_halfraw = data_raw.drop(data_raw[(data_raw['ValidPP'] == 'N') |    
                                (data_raw['ValidPP'] == 'A') & (data_raw.Precipitacion == (-99.9))].index)
len(data_raw)-len(data_halfraw)==498725

True

True

In [32]:
data_halfraw[(data_halfraw.ValidPP=='A') & (data_halfraw.Fecha>'2018-01-01')]

meteodata[(meteodata.Estacion=='OLIVA, AJUNTAMENT') & (meteodata.Tipo=='INMman') &
          (meteodata.Fecha>'2018-01-20') & (meteodata.Fecha<'2018-01-31')]

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
772098,8472E,2006-05-03,-99.9,A,BARRACAS,INMman,696622.0,4431608.0,TE,3.0,NaN,NaN,NaN,NaN
772125,8472E,2006-05-30,-99.9,A,BARRACAS,INMman,696622.0,4431608.0,TE,3.0,NaN,NaN,NaN,NaN
772231,8472E,2006-09-13,-99.9,A,BARRACAS,INMman,696622.0,4431608.0,TE,3.0,NaN,NaN,NaN,NaN
772240,8472E,2006-09-22,-99.9,A,BARRACAS,INMman,696622.0,4431608.0,TE,3.0,NaN,NaN,NaN,NaN
772250,8472E,2006-11-02,-99.9,A,BARRACAS,INMman,696622.0,4431608.0,TE,3.0,NaN,NaN,NaN,NaN
772394,8472E,2007-04-26,-99.9,A,BARRACAS,INMman,696622.0,4431608.0,TE,3.0,NaN,NaN,NaN,NaN
772773,8472E,2008-05-09,-99.9,A,BARRACAS,INMman,696622.0,4431608.0,TE,3.0,NaN,NaN,NaN,NaN
772775,8472E,2008-05-11,-99.9,A,BARRACAS,INMman,696622.0,4431608.0,TE,3.0,NaN,NaN,NaN,NaN
772781,8472E,2008-05-17,-99.9,A,BARRACAS,INMman,696622.0,4431608.0,TE,3.0,NaN,NaN,NaN,NaN
772803,8472E,2008-06-08,-99.9,A,BARRACAS,INMman,696622.0,4431608.0,TE,3.0,NaN,NaN,NaN,NaN


In [459]:
data_raw.head()#[data_raw.ValidPP != 'S']

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
0,7263,1969-11-01,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
1,7263,1969-11-02,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
2,7263,1969-11-03,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
3,7263,1969-11-04,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
4,7263,1969-11-05,0.0,S,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN


In [43]:
data_halfraw[data_halfraw.ValidPP=='A']

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
11655,7263,2001-10-10,31.8,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
11825,7263,2002-03-29,4.4,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
11838,7263,2002-04-11,24.1,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
12055,7263,2002-11-14,3.5,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
12114,7263,2003-01-12,14.8,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
13666,7263,2007-04-14,1.8,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
15262,7263,2014-11-04,9.0,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
16379,7263,2016-12-17,15.0,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
17183,7263,2013-11-27,31.8,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN
17393,7263,2017-11-29,3.8,A,LA MARINA,INMman,706717.0,4224695.0,A,1.0,NaN,NaN,NaN,NaN


In [44]:
len(data_halfraw)

9397180

### Drop Temp if Not valid

In [90]:
data_halfraw.ValidTmax.unique()

array([nan, 'S', 'N'], dtype=object)

In [97]:
data_halfraw[data_halfraw.ValidTmin=='S'].describe()

,Precipitacion,Xutm,Yutm,Umbral_20_40_km_costa,Tmaxima,Tminima
count,3.016111e+06,3.016111e+06,3.016111e+06,3.016111e+06,3.015213e+06,3.016111e+06
mean,1.374667e+00,7.028651e+05,4.344603e+06,1.876033e+00,2.164323e+01,1.054680e+01
std,6.853257e+00,4.871199e+04,8.333487e+04,8.623239e-01,7.587612e+00,6.644911e+00
min,-4.000000e-01,5.370000e+05,4.141490e+06,1.000000e+00,-1.500000e+01,-2.700000e+01
25%,0.000000e+00,6.808070e+05,4.283759e+06,1.000000e+00,1.600000e+01,6.000000e+00
50%,0.000000e+00,7.094020e+05,4.342519e+06,2.000000e+00,2.150000e+01,1.020000e+01
75%,0.000000e+00,7.331660e+05,4.411822e+06,3.000000e+00,2.800000e+01,1.600000e+01
max,7.900000e+02,8.150770e+05,4.538584e+06,3.000000e+00,5.270000e+01,3.500000e+01


In [98]:
data_halfraw.Tmaxima.describe()

count    3.147875e+06
mean     2.102235e+01
std      8.280157e+00
min     -9.900000e+01
25%      1.550000e+01
50%      2.100000e+01
75%      2.750000e+01
max      5.270000e+01
Name: Tmaxima, dtype: float64

In [99]:
data_halfraw.Tmaxima[data_halfraw['ValidTmax'] == 'N'] = np.nan
data_halfraw.Tmaxima.describe()

C:\Users\ceam\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


count    3.003541e+06
mean     2.168008e+01
std      7.523284e+00
min     -1.500000e+01
25%      1.600000e+01
50%      2.150000e+01
75%      2.800000e+01
max      4.900000e+01
Name: Tmaxima, dtype: float64

In [100]:
data_halfraw.Tminima[data_halfraw['ValidTmin'] == 'N'] = np.nan
data_halfraw.Tminima.describe()

C:\Users\ceam\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


count    3.016111e+06
mean     1.054680e+01
std      6.644911e+00
min     -2.700000e+01
25%      6.000000e+00
50%      1.020000e+01
75%      1.600000e+01
max      3.500000e+01
Name: Tminima, dtype: float64

## Spatial cleaning

In [52]:
# Eliminar datos de estaciones que no sean C, V, A, MU, AB, CU, TE

data_halfraw = data_halfraw[data_halfraw.Provincia.isin(['CU', 'AB','MU','TE','C','V','A'])]

In [83]:
# longitud de cadascun dels items d'una Serie (columna). per saber si uniforme, usar mean()

data_halfraw.Yutm.astype('str').str.len().mean() 

9.0

In [89]:
data_halfraw.Umbral_20_40_km_costa.astype('str').str.len().describe()

count    8661698.0
mean           3.0
std            0.0
min            3.0
25%            3.0
50%            3.0
75%            3.0
max            3.0
Name: Umbral_20_40_km_costa, dtype: float64

## Time series cleaning

In [ ]:
a = data_halfraw.CodigoEst.unique() 
b = data_raw.CodigoEst.unique()

main_list = np.setdiff1d(b,a)
main_list

In [47]:
data_raw.Fecha

<bound method _noarg_wrapper.<locals>.wrapper of <pandas.core.strings.StringMethods object at 0x000001D897ED7898>>

In [24]:
# el format de les dates és uniforme 
#data_halfraw.Fecha = data_halfraw.Fecha.apply(pd.to_datetime)    NO UTILITZAT

KeyboardInterrupt: 

In [78]:
data_halfraw.dtypes

CodigoEst                 object
Fecha                     object
Precipitacion            float64
ValidPP                   object
Estacion                  object
Tipo                      object
Xutm                     float64
Yutm                     float64
Provincia                 object
Umbral_20_40_km_costa    float64
Tmaxima                  float64
ValidTmax                 object
Tminima                  float64
ValidTmin                 object
dtype: object

## Sort values

In [71]:
# len(str(data_halfraw.Fecha[0]))

data_halfraw.Fecha.str.len().mean()

10.0

In [73]:
# By Prov, Name, Date
data_halfraw.sort_values(by=['Provincia','Estacion','Fecha'], inplace=True)
data_halfraw.head()

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
979498,8021A,1975-05-01,0.0,S,"AGOST, ESCOLA",INMman,705962.0,4257098.0,A,1.0,NaN,NaN,NaN,NaN
979499,8021A,1975-05-02,0.0,S,"AGOST, ESCOLA",INMman,705962.0,4257098.0,A,1.0,NaN,NaN,NaN,NaN
979500,8021A,1975-05-03,0.0,S,"AGOST, ESCOLA",INMman,705962.0,4257098.0,A,1.0,NaN,NaN,NaN,NaN
979501,8021A,1975-05-04,2.8,S,"AGOST, ESCOLA",INMman,705962.0,4257098.0,A,1.0,NaN,NaN,NaN,NaN
979502,8021A,1975-05-05,15.9,S,"AGOST, ESCOLA",INMman,705962.0,4257098.0,A,1.0,NaN,NaN,NaN,NaN


In [37]:
data_halfraw.head()

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
count,8661698,8661698,8661698.0,8661698,8661698,8661698,8661698.0,8661698.0,8661698,8661698.0,3147875.0,3147875,3147296.0,3147296
unique,803,38896,NaN,2,798,4,NaN,NaN,7,NaN,NaN,2,NaN,2
top,7102,1994-03-09 00:00:00,NaN,S,ONTUR GRUPO ESCOLAR,INMman,NaN,NaN,V,NaN,NaN,S,NaN,S
freq,30896,462,NaN,8653843,30896,8504806,NaN,NaN,2593294,NaN,NaN,3003541,NaN,3016111
first,NaN,1912-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Subsets

Pequeños paquetes de información para poder guardar en csv:      
data.to_csv(‘2018.csv’ encoding=’utf-8’)

## Condiciones en el rango de datos

### Filtro temporal

In [115]:
data2018 = data_halfraw.Provincia[(data_halfraw.Fecha>'2017-12-31') & (data_halfraw.Fecha<'2019-01-01')].unique()
data2018

array(['A', 'V', 'C'], dtype=object)

### Filtro espacial

In [ ]:
# Eliminar MU i CA

# Coordenades geogràfiques

In [45]:
import utm
utm.to_latlon(702979, 4217290, 30, 'True')

(38.08062739602945, -0.6857104701051528)

In [46]:
utm.to_latlon(705706, 4218407, 30, 'True')


(38.09006996813483, -0.6543205349100778)

--------------------------------------------------

# Input file (processed DB)

In [2]:
path = r'C:/CicloHidrico/Datos'
os.chdir(path)

data = pd.read_csv("meteodata.csv", index_col=0) # DB datos procesados de lluvia y temperatura

C:\Users\ceam\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (1,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\ceam\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


# Analysis

## General info

In [9]:
# Automatic stations : only 15
data.Estacion[data.Tipo=='INMaut'].unique()

array(['BENIDORM, AQUAGEST', 'COCENTAINA, BOMBERS AUTOMATICA',
       'EL PINOS, C.P. S.CATA, AUTOMAT', 'ROJALES, EL MOLINO, AUTOMATICA',
       'VILLENA 2', 'XABIA, AJUNTAMENT, AUTOMATICA',
       'CASTELLFORT, AUTOMATICA', 'TORREBLANCA, AJUNTAMENT',
       'VINAROS, AUMAR, AUTOMATICA', 'YECLA, AUTOMATICA', 'BARX',
       'OLIVA, AJUNTAMENT, AUTOMATICA', 'POLINYA, PISCIFACT, AUTOMATICA',
       'UTIEL, LA CUBERA, AUTOMATICA', 'XATIVA, LES PERERES AUTOMATICA'],
      dtype=object)

## Subsets

In [3]:
data2018 = data[(data.Provincia.isin(['CU', 'AB','TE','V','A']))&
          (data.Fecha>'2017-12-31')& (data.Fecha<'2019-01-01')]    # dejamos fuera Castellón y Murcia, escogemos datos 2018

print('Número de estaciones con datos en 2018 = '+ str(len(data2018.CodigoEst.unique())))
print('Número de províncias con datos en 2018 = '+ str(data2018.Provincia.unique()))
print('Número de estaciones con datos en Valencia = '+ str(len(data2018.CodigoEst[data2018.Provincia=='V'].unique())))
print('Número de estaciones con datos en Alacant = '+ str(len(data2018.CodigoEst[data2018.Provincia=='A'].unique())))

Número de estaciones con datos en 2018 = 164
Número de províncias con datos en 2018 = ['A' 'V']
Número de estaciones con datos en Valencia = 108
Número de estaciones con datos en Alacant = 56


In [4]:
data2018V = data2018[data2018.Provincia=='V']  # 2018 y Valencia

In [5]:
data2018V.head()

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
6085345,8381B,2018-01-01,0.0,S,"ADEMUZ, AGRO",INMman,646722.0,4436034.0,V,3.0,NaN,NaN,NaN,NaN
6085346,8381B,2018-01-02,0.0,S,"ADEMUZ, AGRO",INMman,646722.0,4436034.0,V,3.0,NaN,NaN,NaN,NaN
6085347,8381B,2018-01-03,0.0,S,"ADEMUZ, AGRO",INMman,646722.0,4436034.0,V,3.0,NaN,NaN,NaN,NaN
6085348,8381B,2018-01-04,0.0,S,"ADEMUZ, AGRO",INMman,646722.0,4436034.0,V,3.0,NaN,NaN,NaN,NaN
6085349,8381B,2018-01-05,0.0,S,"ADEMUZ, AGRO",INMman,646722.0,4436034.0,V,3.0,NaN,NaN,NaN,NaN


In [6]:
est2018V = data2018V.drop_duplicates(['CodigoEst'])
est2018V = est2018V['CodigoEst', '']

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
6085345,8381B,2018-01-01,0.0,S,"ADEMUZ, AGRO",INMman,646722.0,4436034.0,V,3.0,NaN,NaN,NaN,NaN
6134169,8330,2018-01-01,0.0,S,ALBAL,INMman,722168.0,4364417.0,V,1.0,NaN,NaN,NaN,NaN
6155753,8302O,2018-01-01,0.0,S,"ALBERIC, SANT JORDI",INMman,709941.0,4333838.0,V,2.0,NaN,NaN,NaN,NaN
6164850,8277U,2018-01-01,0.0,S,"ALCANTARA DE XUQUER,AYUNTAMIEN",INMman,710977.0,4327479.0,V,2.0,NaN,NaN,NaN,NaN
6188639,8404A,2018-01-01,0.0,S,"ALCUBLAS, AYUNTAMIENTO",INMman,696502.0,4407661.0,V,3.0,NaN,NaN,NaN,NaN
6239640,8325A,2018-01-01,0.0,S,"ALGEMESI, COOPERATIVA",INMman,721268.0,4341247.0,V,1.0,NaN,NaN,NaN,NaN
6262188,8328E,2018-01-01,0.0,S,ALGINET,INMman,718174.0,4348874.0,V,1.0,18.8,S,11.6,S
6270260,8328F,2018-01-01,0.0,S,"ALGINET, COAGRI",INMman,719295.0,4349091.0,V,1.0,18.6,S,11.5,S
6347669,8301U,2018-01-01,0.0,S,"ALZIRA, TORRECHO",INMman,723396.0,4337388.0,V,1.0,NaN,NaN,NaN,NaN
6386346,8278E,2018-01-01,0.0,S,"ANTELLA, AGRARIA",INMman,708159.0,4328546.0,V,2.0,NaN,NaN,NaN,NaN


In [7]:
data.head()

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
0,8021A,1975-05-01,0.0,S,"AGOST, ESCOLA",INMman,705962.0,4257098.0,A,1.0,NaN,NaN,NaN,NaN
1,8021A,1975-05-02,0.0,S,"AGOST, ESCOLA",INMman,705962.0,4257098.0,A,1.0,NaN,NaN,NaN,NaN
2,8021A,1975-05-03,0.0,S,"AGOST, ESCOLA",INMman,705962.0,4257098.0,A,1.0,NaN,NaN,NaN,NaN
3,8021A,1975-05-04,2.8,S,"AGOST, ESCOLA",INMman,705962.0,4257098.0,A,1.0,NaN,NaN,NaN,NaN
4,8021A,1975-05-05,15.9,S,"AGOST, ESCOLA",INMman,705962.0,4257098.0,A,1.0,NaN,NaN,NaN,NaN


## pl 